In [ ]:
import numpy as np
import random as rd
import shap
from math import log
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import roc_curve,auc
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import validation_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from mlxtend.plotting import plot_decision_regions
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as pl

In [2]:
def plot_matrix(y_true, y_pred, labels_name, title=None, thresh=0.8, axis_labels=None,fontsize=20,title_fontsize=22, axis_title_fontsize=18):
    # 利用sklearn中的函数生成混淆矩阵并归一化
    cm = metrics.confusion_matrix(y_true, y_pred, labels=labels_name, sample_weight=None)  # 生成混淆矩阵
    print(cm,"\n")
    epsilon = 1e-7
    cm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + epsilon)  # 归一化
    print(cm,"\n")
    cm[np.isnan(cm)] = 1
    print(cm)

    cmp = ConfusionMatrixDisplay(cm, display_labels=np.arange(25))
    fig, ax = plt.subplots(figsize=(20,10))

    im=pl.imshow(cm, interpolation='nearest', cmap=pl.get_cmap('Blues'),vmin=0.0)
    cbar = plt.colorbar(im, ax=ax)  # 获取颜色渐进条对象
    cbar.ax.tick_params(labelsize=axis_title_fontsize)
    # pl.colorbar()  # 绘制图例

    # 图像标题
    if title is not None:
        pl.title(title)
        pl.title(title, fontsize=title_fontsize)
    # 绘制坐标
    num_local = np.array(range(len(labels_name)))
    if axis_labels is None:
        axis_labels = labels_name
    pl.xticks(num_local, axis_labels, rotation=45,fontsize=axis_title_fontsize)  # 将标签印在x轴坐标上， 并倾斜45度
    pl.yticks(num_local, axis_labels,fontsize=axis_title_fontsize)  # 将标签印在y轴坐标上
    pl.ylabel('True label',fontsize=axis_title_fontsize)
    pl.xlabel('Predicted label',fontsize=axis_title_fontsize)

    # 为底部的标签留出更多的空间
    fig.subplots_adjust(bottom=0.4, right=0.85) 

    # 调整子图间距以适应坐标轴标签
    plt.tight_layout()
    
    for i in range(np.shape(cm)[0]):
        for j in range(np.shape(cm)[1]):
            percentage = cm[i][j] * 100
        # 检查百分比是否大于0以避免在文本中显示0%
            if percentage >= 0:
            # 格式化为固定的小数点后两位的百分比字符串
               percentage_str = "{:.0f}%".format(percentage)
            # 根据阈值设置文本颜色
               color = "white" if cm[i][j] > thresh else "black"
            # 在混淆矩阵的相应位置添加文本
               plt.text(j, i, percentage_str,
                        ha="center", va="center",
                        color=color,fontsize=fontsize)
    
    # 保存为矢量图
    plt.savefig("RF_confusion_matrix(T-VMS-5).svg", format="svg")  # 保存为SVG格式的矢量图
    # 显示
    pl.show()


In [3]:
def count_nber(nb,arr):
    return sum(element == nb for element in arr)

In [4]:
def cross_val_class_score(clf, X, y, cv=2):
    kfold = StratifiedKFold(n_splits=cv).split(X, y)
    class_accuracy = []
    for k, (train, test) in enumerate(kfold):
        clf.fit(X[train], y[train])  # 使用训练数据拟合模型
        y_test = y[test]
        y_pred = clf.predict(X[test])
        # 计算混淆矩阵，通过混淆矩阵找出对于每一个折，分类是0或者1的概率
        class_acc = cmat.diagonal() / cmat.sum(axis=1)
        class_accuracy.append(class_acc)
        print('fold: {:d} accuracy {:s}'.format(k + 1, str(class_acc)))
    return np.array(class_accuracy)

In [5]:
def plot_feature_importances(feature_importances, title, feature_names):
    # Normalize the importance values
    feature_importances = 100.0 * (feature_importances / max(feature_importances))

    # Sort the values and flip them
    index_sorted = np.flipud(np.argsort(feature_importances))
    
    print(index_sorted,type(index_sorted))
    # Arrange the X ticks
    pos = np.arange(index_sorted.shape[0]) + 0.5
    # Plot the bar graph
    plt.figure(figsize=(25,8),dpi=80)
    plt.bar(pos, feature_importances[index_sorted], align='center')
    print(feature_names[index_sorted].values.tolist())
    plt.xticks(pos, feature_names[index_sorted].values.tolist()[0]) 
    plt.ylabel('Relative Importance')
    plt.title(title)
    plt.show()

In [ ]:
data = pd.read_excel('T-VMS.xlsx')
X1 = pd.DataFrame(data)
# X1.columns = ['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'MnO', 'MgO', 'CaO', 
# 'Na2O', 'K2O', 'P2O5', 'Sc', 'V', 'Cr', 'Co', 'Ni', 'Cu', 'Zn', 
# 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm',
#  'Eu', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'Ga', 'Gd', 
#  'Pb', 'Th', 'U','Deposit_Type']
display(data)


# 读取盲测集数据
blind_test_data = pd.read_excel('盲测集.xlsx')

display(blind_test_data)

sm = SMOTE(random_state=40)
X_res, y_res = sm.fit_resample(data.iloc[:,:-1],data.iloc[:,-1])
X_pre_val,y_pre_val=(blind_test_data.iloc[:,:-1],blind_test_data.iloc[:,-1])
display(X_res,y_res)
display(X_pre_val,y_pre_val)
X_train, X_test, Y_train, Y_test = train_test_split(X_res,  y_res,
                                                    test_size=0.3, random_state=30, stratify = y_res)
np.random.seed(1)
# clf = RandomForestClassifier(n_estimators=400)

max_depths: list[int] = [3, 4, 5, 6, 7, 9, 11, 13, 15, 17, 19]

train_scores, test_scores = validation_curve(
    estimator=clf,
    X=X_train,
    y=Y_train,
    param_name='max_depth',
    param_range=max_depths,
    cv=5
)

In [7]:
def plot_validation_curve(train_scores, test_scores,
                          param_range, xlabel='', log=False):
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    fig = plt.figure()
    plt.plot(param_range, train_mean,
             color=sns.color_palette('Set1')[1], marker='o',
             markersize=5, label='training accuracy')

    plt.fill_between(param_range, train_mean + train_std,
                     train_mean - train_std, alpha=0.15,
                     color=sns.color_palette('Set1')[1])

    plt.plot(param_range, test_mean,
             color=sns.color_palette('Set1')[0], linestyle='--',
             marker='s', markersize=5,
             label='validation accuracy')

    plt.fill_between(param_range,
                     test_mean + test_std,
                     test_mean - test_std,
                     alpha=0.15, color=sns.color_palette('Set1')[0])

    if log:
        plt.xscale('log')
    plt.legend(loc='lower right')
    if xlabel:
        plt.xlabel(xlabel)
    plt.ylabel('Accuracy')
    plt.ylim(0.9, 1.0)
    return fig

In [ ]:
plot_validation_curve(train_scores, test_scores, max_depths, xlabel='max_depth')
plt.xlim(3, 19)
plt.ylim(0, 1)

In [ ]:
# plt.savefig(validation_curve_path, bbox_inches='tight', dpi=300)
np.random.seed(1)
#clf = tree.DecisionTreeClassifier() #决策树
clf = RandomForestClassifier(n_estimators=300, max_depth=9) #随机森林

clf.fit(X_train, Y_train)
#原测试集
pred = clf.predict(X_test)

#盲测集
pred_test=clf.predict(X_pre_val)


# cmat = confusion_matrix(Y_test, pred)
# cmat = metrics.confusion_matrix(y_pre_val, pred_test)

columns=["Biomdal-Felsic",
"Biomdal-Mafic",
"Felsic-Siliciclastic",
"Mafic",
"Mafic-Siliciclastic"
]
index =  [0,1,2,3,4]
#原绘制混淆矩阵
# plot_matrix(Y_test, pred,index, title='RF_Confusion_matrix',
#                     axis_labels=columns)

# 盲测集绘制混淆矩阵
plot_matrix(y_pre_val,pred_test,index, title='RF_Confusion_matrix',
                    axis_labels=columns)


In [ ]:
#11  运用两个特征变量绘制随机森林算法决策边界图
X2 = X1.iloc[:,:-1].iloc[:, [7,12]]#仅选取workyears、debtratio作为特征变量
y = X1.iloc[:,-1].astype(int)
display(X2,y)
model = RandomForestClassifier(n_estimators=300, max_features=1, random_state=1)
model.fit(X2,y)
model.score(X2,y)
plot_decision_regions(np.array(X2), np.array(y),model)
plt.xlabel('debtratio')#将x轴设置为'debtratio'
plt.ylabel('workyears')#将y轴设置为'workyears'
plt.title('Decision boundary')#将标题设置为'随机森林算法决策边界'
plt.show()
#plt.savefig('随机森林算法决策边界.png')

In [ ]:
housing_data = X1.columns
housing_data1= pd.DataFrame(housing_data).T
plot_feature_importances(clf.feature_importances_, 'Random forest', housing_data1)

In [ ]:

# 假设 clf 是已经训练好的模型，X_train 和 X_pre_val 是训练和测试数据集
# 假设 index 和 columns 已经定义，且 X1.columns 包含了特征名称

# 使用 TreeExplainer 解释模型
explainer_tree = shap.TreeExplainer(clf)
# shap_values_tree = explainer_tree.shap_values(X_pre_val)

# 确保 shap_values_tree 是一个列表，并且每个元素的行数与 X_pre_val 相匹配
if isinstance(shap_values_tree, list):
    # 创建一个新的图形对象，并设置适当的大小
    fig, ax = plt.subplots(figsize=(8, 6))
    # 只显示排名前10的元素
    shap.summary_plot(shap_values_tree, X_pre_val, plot_type="bar", class_inds=index, class_names=columns, max_display=10, show=False)
    # 保存图像
    plt.savefig("RF_SHAP_VALUE(T-VMS-5-test).svg", format="svg", bbox_inches='tight')  # 保存为SVG格式的矢量图，并调整边界框以适应图像
else:
    print("Error: shap_values_tree is not a list.")

# 使用 KernelExplainer 解释模型
explainer_kernel = shap.KernelExplainer(clf.predict, X_train)
# shap_values_kernel = explainer_kernel.shap_values(X_pre_val, nsamples=200)

# 确保 shap_values_kernel 是一个二维数组，并且行数与 X_pre_val 相匹配
if len(shap_values_kernel) == len(X_pre_val):
    # 创建一个新的图形对象，并设置适当的大小
    fig, ax = plt.subplots(figsize=(8, 6))
    # 只显示排名前10的元素
    shap.summary_plot(shap_values_kernel, X_pre_val, feature_names=X1.columns, max_display=10, show=False)
    # 保存图像
    plt.savefig("RF_SHAP_VALUE(T-VMS-5-test).svg", format="svg", bbox_inches='tight')  # 保存为SVG格式的矢量图，并调整边界框以适应图像
else:
    print("Error: shap_values_kernel does not match the number of rows in X_pre_val.")


In [ ]:
shap.summary_plot(shap_values,X_test,feature_names=housing_data )
print(X_test.columns)

#### from sklearn.metrics import accuracy_score # 引入准确度评分函数
y_pred = clf.predict(X_test)
print('训练集模型分数:', clf.score(X_train,Y_train))
print('测试集模型分数:', clf.score(X_test,Y_test))
print("训练集准确率: %.3f" % accuracy_score(Y_train, clf.predict(X_train)))
print("测试集准确率: %.3f" % accuracy_score(Y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_curve,auc
n_classes = len(columns)
df = pd.DataFrame()
pre_score = clf.predict_proba(X_test)
display(pd.DataFrame(pre_score))
df['y_test'] = Y_test.to_list()
name={}
for i in range(n_classes):
    variable_name = "pre" + str(i+1)
    name.setdefault(variable_name, [])
    df['pre_score'+str(i)] = pre_score[:,i]
    
    name["pre" + str(i+1)] = df['pre_score'+str(i)]
display(df,name)

In [18]:
y_list = df['y_test'].to_list()
#pre_list=[pre1,pre2,pre3,pre4,pre5,pre6,pre7]
 
lable_names=['1','2','3','4','5']
colors1 = ["r","b","g",'gold','pink']
colors2 = "skyblue"# "mistyrose","skyblue","palegreen"
my_list = []
linestyles =["-", "--", ":","-", "--"]


In [ ]:
plt.figure(figsize=(12,8),facecolor='w')
for i in range(5):
    roc_auc = 0
     #添加文本信息
    fpr, tpr, threshold = roc_curve(y_list,name["pre" + str(i+1)],pos_label=i)
        # 计算AUC的值
    roc_auc = auc(fpr, tpr)
    print(roc_auc,fpr,tpr)
    plt.text(0.3, 0.05+i*0.1, columns[i]+lable_names[i]+' :ROC curve (area = %0.2f)' % roc_auc)
   
    my_list.append(roc_auc)
    # 添加ROC曲线的轮廓
    plt.plot(fpr, tpr, color = colors1[i],linestyle = linestyles[i],linewidth = 3,
              label = columns[i]+lable_names[i])  #  lw = 1,
    #绘制面积图
   # plt.stackplot(fpr, tpr, color=colors2, alpha = 0.5,edgecolor = colors1[i]) #  alpha = 0.5,
   
# 添加对角线
#plt.plot([0, 1], [0, 1], color = 'black', linestyle = '--',linewidth = 3)
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.grid()
plt.legend()
plt.title("ROC")
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 计算精确度
precision = precision_score(y_pre_val, pred_test, average='macro')
print(f'Precision: {precision}')

# 计算召回率
recall = recall_score(y_pre_val, pred_test, average='macro')
print(f'Recall: {recall}')

# 计算 F1 分数
f1 = f1_score(y_pre_val, pred_test, average='macro')
print(f'F1 Score: {f1}')
